In [1]:
from pathlib import Path
import shutil
from tqdm import tqdm

#from kilosort.io import load_probe
from kilosort import run_kilosort, DEFAULT_SETTINGS

SAVE_PATH = Path('/ix1/pmayo/OneDrive/DATA/kendra_scrappy_0097a/kendra_scrappy_0097a.bin')

In [2]:
import numpy as np

n_chan = 24
chanMap = np.arange(n_chan)
kcoords = np.zeros(n_chan)
xc = np.ones(n_chan)*7.5
yc = np.array([7.5 + 150*i for i in range(n_chan)])

probe = {
    'chanMap': chanMap,
    'xc': xc,
    'yc': yc[::-1],
    'kcoords': kcoords,
    'n_chan': n_chan
}

print(probe)

{'chanMap': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23]), 'xc': array([7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5,
       7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5]), 'yc': array([3457.5, 3307.5, 3157.5, 3007.5, 2857.5, 2707.5, 2557.5, 2407.5,
       2257.5, 2107.5, 1957.5, 1807.5, 1657.5, 1507.5, 1357.5, 1207.5,
       1057.5,  907.5,  757.5,  607.5,  457.5,  307.5,  157.5,    7.5]), 'kcoords': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]), 'n_chan': 24}


In [3]:
#probe = load_probe('/Users/kendranoneman/Projects/mayo/helperfunctions/mappings/probes/N19415.prb')

settings = DEFAULT_SETTINGS
print(settings)
settings['probe'] = probe
settings['n_chan_bin'] = 24
settings['fs'] = 30000
settings['data_dir'] = SAVE_PATH.parent

# preprocessing
settings['nskip']

# spike detection
settings['batch_size'] = 60000*2 # 60000
settings['nblocks'] = 0 # 1
settings['Th_universal'] = 8 # 9
settings['Th_learned'] = 8   # 8

# clustering
settings['min_template_size'] = 10 #10
settings['nearest_templates'] = 23 # 100
#settings['max_channel_distance'] = 16 # None

kilosort_path = SAVE_PATH.parent / 'kilosort4'
temp_wh_path = SAVE_PATH.parent / 'temp_wh.dat'

if kilosort_path.exists() and kilosort_path.is_dir():
    shutil.rmtree(kilosort_path)

if temp_wh_path.exists() and temp_wh_path.is_file():
    temp_wh_path.unlink()    

{'n_chan_bin': 385, 'fs': 30000, 'batch_size': 60000, 'nblocks': 1, 'Th_universal': 9, 'Th_learned': 8, 'tmin': 0, 'tmax': inf, 'nt': 61, 'shift': None, 'scale': None, 'artifact_threshold': inf, 'nskip': 25, 'whitening_range': 32, 'highpass_cutoff': 300, 'binning_depth': 5, 'sig_interp': 20, 'drift_smoothing': [0.5, 0.5, 0.5], 'nt0min': None, 'dmin': None, 'dminx': 32, 'min_template_size': 10, 'template_sizes': 5, 'nearest_chans': 10, 'nearest_templates': 100, 'max_channel_distance': 32, 'max_peels': 100, 'templates_from_data': True, 'n_templates': 6, 'n_pcs': 6, 'Th_single_ch': 6, 'acg_threshold': 0.2, 'ccg_threshold': 0.25, 'cluster_downsampling': 20, 'x_centers': None, 'duplicate_spike_ms': 0.25, 'position_limit': 100}


In [6]:
ops, st, clu, tF, Wall, similar_templates, is_ref, est_contam_rate, kept_spikes = \
    run_kilosort(settings=settings, probe=probe)

kilosort.run_kilosort: Kilosort version 0.1.dev1164+gdfaaf6b.d20240502
kilosort.run_kilosort: Python version 3.10.14
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: System information:
kilosort.run_kilosort: Linux-3.10.0-1160.71.1.el7.x86_64-x86_64-with-glibc2.17 x86_64
kilosort.run_kilosort: x86_64
kilosort.run_kilosort: Using GPU for PyTorch computations. Specify `device` to change this.
kilosort.run_kilosort: Using CUDA device: NVIDIA A100-PCIE-40GB 39.38GB
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: Sorting /ix1/pmayo/OneDrive/DATA/kendra_scrappy_0097a/kendra_scrappy_0097a.bin
kilosort.run_kilosort: Interpreting binary file as default dtype='int16'. If data was saved in a different format, specify `data_dtype`.
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: N samples: 134174700
ki

In [7]:
from kilosort.io import save_preprocessing, load_ops

# NOTE: This will only create the .dat file, it will *NOT* update options for Phy.
#       If you want to use this with Phy, you will need to modify `params.py`
#       in the results directory to point to this new file. Additionally,
#       you must set `hp_filtered=True` and `dtype='int16'`.
ops_path = SAVE_PATH.parent / 'kilosort4' / 'ops.npy'
ops = load_ops(ops_path)
save_preprocessing(SAVE_PATH.parent / 'temp_wh.dat', ops, bfile_path=SAVE_PATH)

kilosort.io :  
kilosort.io : ========================================
kilosort.io : Saving drift-corrected copy of data to: /ix1/pmayo/OneDrive/DATA/kendra_scrappy_0097a/temp_wh.dat...
kilosort.io : Writing batch 0/1119...
kilosort.io : Writing batch 100/1119...
kilosort.io : Writing batch 200/1119...
kilosort.io : Writing batch 300/1119...
kilosort.io : Writing batch 400/1119...
kilosort.io : Writing batch 500/1119...
kilosort.io : Writing batch 600/1119...
kilosort.io : Writing batch 700/1119...
kilosort.io : Writing batch 800/1119...
kilosort.io : Writing batch 900/1119...
kilosort.io : Writing batch 1000/1119...
kilosort.io : Writing batch 1100/1119...
kilosort.io : ========================================
kilosort.io : Copying finished.
kilosort.io :  
